# Term Project Progress Notebook - Manipulation and Analysis (NEW CONTINUING)

- Lindsey Rojtas
- LING1340
- ler75@pitt.edu

Welcome to the second Notebook! It should be noted that this is a **continuation** of `progress_data`! My progress report 3 starts there and will end here. 

## PROGRESS REPORT 3 CONTINUED

Before anything else, we should load in those pickle files. 

In [1]:
import pandas as pd

In [2]:
byfile = pd.read_pickle('../privdata/byfile_df.pkl')
bypar = pd.read_pickle('../privdata/byparticipant_df.pkl')
wordlist = pd.read_pickle('../privdata/wordlist_df.pkl')

In [3]:
byfile.head()

,file,participants,content
0,data001.txt,"[F107, F023, M023, F128]",[F107：＊＊＊の町というのはちいちゃくって、城壁がこう町全体をぐるっと回ってて、それが城...
1,data002.txt,"[F107, F023, F128]","[F107：今度はーイギリスにもアメリカと同様のテロが起こるだろうって言ったんだってよ。, ..."
2,data003.txt,"[F033, F056]","[F033：倒れちゃう。, F056：いきなり倒れた。, F033：どうしよう。あっ、この間..."
3,data004.txt,"[M018, F128]",[F128：いや、別にいいよ。ローソンでいいやろ。ちょっと倒していい、これ。どうよ、調子は。...
4,data005.txt,"[M023, F128, F116, M026]","[F128：来てたときによく貸してもらったやつだ。, M023：そう、そんな感じのとこ。, ..."


In [4]:
bypar.head()

,participant,age,appears_count,appears_in,content
0,F001,Early 20s,5,"[data105.txt, data086.txt, data076.txt, data07...",うーん、わかんない。そういうこと言わないで。うるさいな。うるさい。うるさいって言ってるの。う...
1,F002,Late 60's,3,"[data033.txt, data032.txt, data031.txt]",２７歳から現在まで東京都に居住。南仏へいらしたそうだけど、（うん）どうでしたか。みんな、太っ...
2,F003,Late 80's,1,[data129.txt],そうねえ。＜笑い＞先生はね、師範卒業したのがねえ、１９歳だったのよ。だーから、若い先生でね。...
3,F004,Late 20's,14,"[data096.txt, data094.txt, data092.txt, data08...",うん、まあね。はい、もう始まってますからね。よろしくね。ちょっとちょっと、ちゃんとさ、つなぐ...
4,F005,Late 20's,3,"[data052.txt, data023.txt, data015.txt]",はーい。いや、F034さんってー、やっぱりー、ハンバーガーとか好きですよねー。＊はなしを＊。...


In [5]:
wordlist.head()

,katakana,translation,frequency
0,パーセント,percent,63392
1,アメリカ,America,28243
2,ページ,page,24642
3,センター,center,20664
4,サービス,service,16630


### Exploratory Analysis and Developing Questions
So, let's get some structure in here. What I'm gonna do next is check the length of everything as well as check for some potential issues that will come about in our analysis. 

In [6]:
len(wordlist)

5192

So there are around 5,000 entries in our list of Katakana words.... 

In [7]:
len(bypar)

197

... 197 participants... 

In [8]:
len(byfile)

129

... who spoke in 129 files. It's pretty easy to see that not everyone spoke in the same amount of files for the same amount of time, so we should examine the data for potential outliers. One thing I can think of off the top of my head is how many times a certain participant was present in the conversations. We can see that F004 was present in 14 different files, so obviously she will have more conversation data than anyone else, unless her contributions in many of the files were marginal. Let's take a peek at some value counts:

In [9]:
bypar['appears_count'].value_counts()

1     161
2      16
3       7
5       4
4       4
7       2
14      1
11      1
6       1
Name: appears_count, dtype: int64

Ah, so we're seeing a problem here. The majority of the participants only appeared in one file, but there are quite a few outliers here. I could just exclude them from my analysis from the get-go, but we should probably see just how long the strings of text are for each participant. 

In [10]:
bypar['length'] = 0
for i in range(len(bypar)):
    bypar['length'][i] = len(bypar['content'][i])

C:\Users\lindsey rojtas\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
bypar.head()

,participant,age,appears_count,appears_in,content,length
0,F001,Early 20s,5,"[data105.txt, data086.txt, data076.txt, data07...",うーん、わかんない。そういうこと言わないで。うるさいな。うるさい。うるさいって言ってるの。う...,36989
1,F002,Late 60's,3,"[data033.txt, data032.txt, data031.txt]",２７歳から現在まで東京都に居住。南仏へいらしたそうだけど、（うん）どうでしたか。みんな、太っ...,27372
2,F003,Late 80's,1,[data129.txt],そうねえ。＜笑い＞先生はね、師範卒業したのがねえ、１９歳だったのよ。だーから、若い先生でね。...,5615
3,F004,Late 20's,14,"[data096.txt, data094.txt, data092.txt, data08...",うん、まあね。はい、もう始まってますからね。よろしくね。ちょっとちょっと、ちゃんとさ、つなぐ...,133395
4,F005,Late 20's,3,"[data052.txt, data023.txt, data015.txt]",はーい。いや、F034さんってー、やっぱりー、ハンバーガーとか好きですよねー。＊はなしを＊。...,24084


In [12]:
bypar['length'].describe()

count       197.000000
mean      12055.725888
std       14839.251277
min         458.000000
25%        5683.000000
50%        8206.000000
75%       12992.000000
max      133395.000000
Name: length, dtype: float64

So there are some wildly different levels of contribution here. We see that F004 has way more contributions than even the 75th percentile - this is not surprising, but it is something we really need to take into account if we find that she used a ton of Katakana. A way we could approach this is take a ratio of Katakana characters to overall characters. This, though, could be an issue because onomatopoeia words are also written in Katakana. It may be something worth trying, however. Another option I could try is just dropping anyone that participated in one or two conversations. I might try both (or more things!) in an attempt to see how the results differ -- maybe younger people use more onomatopoeia than older people too!

Now, let's test out some methods of gathering up how often these words are used in these files. Below is a line of code I used to see on what level in a dataframe a word would be detected:

In [13]:
word = 'アメリカ' # "amerika" = america
word in byfile['content'][1][0]

True

We can see here that it's probably a good thing that we kept everyone's utterances in one big long string, because that eliminates one more level of a data hierarchy to go into.  

In [14]:
ct = 0
for i in range(len(bypar)):
    if word in bypar['content'][i]:
        ct += 1
print(ct)

69


So this means that the word for America appears 69 times in this dataset. Let's try another common word:

In [15]:
word = "パーセント" # paasento = percent
ct = 0
for i in range(len(bypar)):
    if word in bypar['content'][i]:
        ct += 1
print(ct)

3


In [16]:
word = "テレビ" # terebi = television
ct = 0
for i in range(len(bypar)):
    if word in bypar['content'][i]:
        ct += 1
print(ct)

76


So the word for percent only shows up 3 times, but the word for television shows up 76 times... this makes sense since the word "percent" doesn't typically appear in casual conversation but "television" does. It seems likely that the frequencies in `wordlist` and the frequencies as far as how often these things actually appear in conversation. We could probably save some time from further analysis if we get rid of words that don't appear at all in these conversations. Let's do that now. 

In [17]:
wordlist['conv_freq'] = ''

In [18]:
for i in range(len(wordlist)):
    word =  wordlist['katakana'][i]
    ct = 0
    for j in range(len(bypar)):
        if word in bypar['content'][j]:
            ct += 1
    if ct == 0:
        wordlist['conv_freq'][i] = None
    else:
        wordlist['conv_freq'][i] = ct

C:\Users\lindsey rojtas\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lindsey rojtas\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
wordlist.head(15)

,katakana,translation,frequency,conv_freq
0,パーセント,percent,63392,3
1,アメリカ,America,28243,69
2,ページ,page,24642,27
3,センター,center,20664,33
4,サービス,service,16630,21
5,システム,system,16458,17
6,メートル,metre,15960,17
7,テレビ,television,15644,76
8,メール,mail,15589,72
9,データ,data,13210,28


Again, the frequency column doesn't reflect how frequently these words appear in these conversation files. Let's drop words we don't see. 

In [20]:
wordlist = wordlist.dropna()

In [21]:
wordlist = wordlist.reset_index(drop=True)

In [22]:
wordlist.head(15)

,katakana,translation,frequency,conv_freq
0,パーセント,percent,63392,3
1,アメリカ,America,28243,69
2,ページ,page,24642,27
3,センター,center,20664,33
4,サービス,service,16630,21
5,システム,system,16458,17
6,メートル,metre,15960,17
7,テレビ,television,15644,76
8,メール,mail,15589,72
9,データ,data,13210,28


In [23]:
len(wordlist)

2734

We cut our amount of words in half, wow. I wonder what the most frequently used word in these conversations was:

In [24]:
wordlist['conv_freq'].max()

193

In [25]:
wordlist['translation'][pd.Index(wordlist['conv_freq']).get_loc(193)]

'la'

So here's another issue: the word for "la" (ラ) is probably only so frequent because it has the potential to appear in other words. 

In [26]:
for i in range(len(wordlist)):
    if 'ラ' in wordlist['katakana'][i]:
        print(wordlist['katakana'][i], end = ', ')

フランス, クラブ, ライン, カメラ, バランス, クラス, プログラム, ガラス, ドラマ, カラー, ブランド, ボランティア, プラス, ラジオ, レストラン, トラブル, ライブ, ドライブ, プラン, トラック, ラーメン, サラダ, イスラム, イラク, グラス, ラブ, オランダ, グラフ, ラン, オーストラリア, ランキング, ドライバー, ブラック, サラリーマン, ライフ, ランド, ブラジル, ランチ, キャラクター, ライト, キャラ, イスラエル, イラスト, プラスチック, オペラ, イラン, アラブ, ラッキー, ランプ, リラックス, グランド, コラム, ラベル, ライバル, ブラシ, ランク, ラウンド, ライター, ライオン, クラシック, グラウンド, ライト, ラスト, プライド, プライバシー, エラー, ベランダ, カメラマン, フライ, ドラゴン, リストラ, ウラン, マラソン, プライベート, ブラウン, ベテラン, ラ, ライダー, ラップ, ラテン, アラビア, コーラ, ミネラル, ドライ, ポーランド, マフラー, オーケストラ, ドラム, ライス, ピラミッド, フランク, ディーラー, クーラー, フォーラム, フラワー, フラッシュ, レギュラー, テラス, ラリー, ニュージーランド, プラント, ライセンス, ブラウス, サングラス, トランク, ライフスタイル, カウンセラー, セラー, ラッシュ, トライ, スライス, ストラップ, ミラノ, プラグ, モラル, オーラ, スコットランド, ミラー, キャラメル, ライト, フィンランド, ラグビー, カリキュラム, ランニング, マスカラ, フライト, アラン, バニラ, フラン, ララ, ローラ, カラフル, ポピュラー, ラフ, ブラン, ミラー, アトラクション, ラグ, ブラザー, ストライプ, ライナー, ミイラ, ストライク, プロペラ, フラ, ゴリラ, サウジアラビア, ラオス, ブラウン, ライ, トランプ, マニラ, ラット, スラム, トラスト, パノラマ, ドラフト, セントラル, ラム, ブラッド, ヒマラヤ, レスラー, ローラー, ブラインド, グラタン, シンデレラ, キラー, アイランド, セラピー, イ

That's a lot of words that are being counted as another usage of "la." Maybe we should look at all "words" that are only one character in length and drop those too. 

In [27]:
for i in range(len(wordlist)):
    if len(wordlist['katakana'][i]) == 1:
        wordlist['katakana'][i] = None

C:\Users\lindsey rojtas\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
wordlist = wordlist.dropna()

In [29]:
wordlist = wordlist.reset_index(drop=True)

In [30]:
len(wordlist)

2722

Only a couple words dropped, but it'll make a big difference later on. Let's check the most frequent word again:

In [31]:
wordlist['conv_freq'].max()

138

In [32]:
wordlist['translation'][pd.Index(wordlist['conv_freq']).get_loc(138)]

'strike'

This is pretty neat! I wouldn't have expected this to be the most common word, but then again, I don't know what the conversation topics are. I'd investigate less-common words, but that's probably quite a few words. As I was debugging, I found a lot of words that only appeared once, but appeared nonetheless. It also seemed like my cutoff point back in progress report 1 was a good one; there were quite a few zero-frequency words towards the end of my list (part of my debugging process was to put print statements everywhere... not much else in the process other than trying to explain the code to my blissfully ignorant cat). 
  
  
I do want to check out the length of these words and compare it to their frequencies. I noted in the last progress report that a shortened version of the word for "centimeter" is used more frequently than the full version, so I'm going to investigate the relationship between length (both in Katakana characters and English translation) and usage, if there is any relationship to begin with. This is a question I've wanted to investigate since the beginning, before I even came up with the idea of comparing Katakana use to age. 

In [33]:
def getlen(w):
    ln = len(w)
    return ln

In [34]:
wordlist['kata_len'] = wordlist['katakana'].apply(getlen)
wordlist['tln_len'] = wordlist['translation'].apply(getlen)

In [35]:
wordlist.head()

,katakana,translation,frequency,conv_freq,kata_len,tln_len
0,パーセント,percent,63392,3,5,7
1,アメリカ,America,28243,69,4,7
2,ページ,page,24642,27,3,4
3,センター,center,20664,33,4,6
4,サービス,service,16630,21,4,7


In [36]:
wordlist['kata_len'].min()

2

In [37]:
wordlist['tln_len'].min()

1

In [44]:
wordlist['translation'][pd.Index(wordlist['kata_len']).get_loc(2)]

18         dollar
28            bus
33           kilo
35           door
39            one
          ...    
2575          yak
2577    agitation
2578         than
2589          jig
2603            I
Name: translation, Length: 343, dtype: object

In [45]:
wordlist['translation'][pd.Index(wordlist['tln_len']).get_loc(1)]

377     H
2603    I
Name: translation, dtype: object

So as I was poking around, I noticed that even more of my words that I included should probably be dropped -- ones that don't seem to have been borrowed from English (外国 = foreign country). I really only want to focus on English borrowings, so let's just drop those real quick:

In [39]:
for i in range(len(wordlist)):
    if wordlist['translation'][i] == '外国' or wordlist['translation'][i] == '四':
        wordlist['translation'][i] = None

C:\Users\lindsey rojtas\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
wordlist = wordlist.dropna()

In [41]:
wordlist = wordlist.reset_index(drop=True)

In [42]:
len(wordlist)

2612

We still have a pretty sizable list here, despite all that we dropped. There are some other considerations we should take here, like how the translation for "I" can also be used as a name or a word for "love", but I don't know how the word is being used in the conversation, so I'm going to leave it in.   
  
